In [ ]:
import pandas as pd, numpy as np, datetime as dt, sys, os, warnings
from calendar import month_name as month_names

warnings.simplefilter('ignore')

# functions for processing

def get_marques_at_ones(df):
    marques = []
    for marque in df.Marque:
        if marque not in marques:
            marques.append(marque)
            
    return marques

def get_mois_at_ones():
    mois = [m for m in range(1,13)]
    return mois

def get_segs_at_ones(df):
    segs = []
    for seg in df.segmentation:
        if seg not in segs:
            segs.append(seg)
            
    return segs

def get_Model_segmentations(df):
    data = {}
    for i in range(df.shape[0]):
        key = df.iloc[i]['Modèle AIVAM']
        val = df.iloc[i]['Segmentation']
        data[str(key)] = val
        
    return data

def get_hyb_type_mod(df):
    data = {}
    for i in range(df_aivam_hyb_type_mod.shape[0]):
        key = df.iloc[i]['Modèlel']
        val = df.iloc[i]['Type Hyb']
        data[str(key)] = val
        
    return data

def get_mode_CSUV_transmission(df):
    data = {}
    for i in range(df_aivam_transmission.shape[0]):
        key = df.iloc[i]['Modèlel']
        val = df.iloc[i]['Transmission']
        data[key] = val
        
    return data

def get_seg(seg, x):
    if str(x) not in seg.keys():
        return 'None'    
    return seg[str(x)]

def get_hyb(hyb, x):
    if str(x['Modèle AIVAM']) not in hyb.keys():
        return x['CARBURANT']
    return hyb[str(x['Modèle AIVAM'])]

def get_CSUV_transmission(transmission, x):
    if x not in transmission.keys():
        return 'None'
    return transmission[x]

def get_sales_perCent_by_key(df, key = 'segmentation'):
    keys = list(df[key].value_counts().index.values)
    key_val = {}
    for key_ in keys:
        x = df[(df[key] == key_)]['Sales'].sum() 
        key_val [key_] = x

    return key_val

def get_groupe(x):
    marques = ['PEUGEOT', 'CITROEN', 'DS', 'OPEL', 'FIAT', 
        'JEEP', 'ALFA ROMEO', 'ABARTH'
    ]
    if x in marques:
        return 'STELLANTIS'
    else:
        return 'other marques'

def get_Brand_order(x):
    marques = ['PEUGEOT', 'CITROEN', 'DS', 'OPEL', 'FIAT', 
        'JEEP', 'ALFA ROMEO', 'ABARTH'
    ]
    if x in marques:
        index_ = marques.index(x) + 1
        return index_
    else:
        return 0


def get_Unnamed_columns(df):
    l = []
    for i in df.columns:
        if i[:7] == 'Unnamed':
            l.append(i)
    return l

def get_model_net(x):
    x = str(x).strip()
    if x in models_dict.keys():
        return models_dict[x]
    else:
        return x

def clean_cities(x):
    x = x.strip()
    if x[-8:] == 'nimellal':
        return 'Benimellal'
    elif x[-5:] == 'nitra':
        return 'Kenitra'
    elif x[:4] == 'Mekn':
        return 'Meknes'
    elif x[:3] == 'Sal':
        return 'Sale'
    else:
        return x
    
def global_process(df_aivam_brut, df_aivam_segments, df_aivam_hyb_type_mod, df_aivam_transmission):
        
    model_segments = get_Model_segmentations(df_aivam_segments)
    df_aivam_brut['segmentation'] = df_aivam_brut['Modèle AIVAM'].apply(lambda x: 
                                                                        get_seg(model_segments, x))
    
    hyb_type_mod = get_hyb_type_mod(df_aivam_hyb_type_mod)
    df_aivam_brut['Type Motorisation'] = df_aivam_brut.apply(lambda x: 
                                                            get_hyb(hyb_type_mod, x), axis = 1)
    
    mode_CSUV_transmission = get_mode_CSUV_transmission(df_aivam_transmission)
    
    df_aivam_brut['Transmission'] = df_aivam_brut['Modèle AIVAM'].apply(lambda x: 
                                                                        get_CSUV_transmission(mode_CSUV_transmission, x))
    
    df_aivam_brut['Months'] = df_aivam_brut['Mois'].apply(lambda x: month_names[x])

    df_aivam_brut['Sales'] = df_aivam_brut['Unités']

    df_aivam_brut['City'] = df_aivam_brut['Ville']

    df_aivam_brut['City'] = df_aivam_brut['City'].apply(
        lambda x: clean_cities(x)
    )

    df_aivam_brut['Groupe'] = df_aivam_brut['Marque'].apply(
        lambda x: 
        get_groupe(x)
    )

    df_aivam_brut['Brand Order'] = df_aivam_brut['Marque'].apply(
        lambda x: 
        get_Brand_order(x)
    )

    df_aivam_brut['Model Net'] = df_aivam_brut['Modèle AIVAM'].apply(
        lambda x: get_model_net(x)
    )

    return df_aivam_brut

def rename_year(df):
    if 'Annee' in df.columns:
        df = df.rename(columns = {
            'Annee': 'Year'
        })
    elif 'Année' in df.columns:
        df = df.rename(columns = {
            'Année': 'Year'
        })
    return df


def merge_Vp_Vul(df_name):
    df = pd.ExcelFile(df_name)
    df_vp = pd.read_excel(df_name, sheet_name = df.sheet_names[0], header = [2])
    df_vul = pd.read_excel(df_name, sheet_name = df.sheet_names[1], header = [2])

    df_res = pd.concat([df_vp, df_vul])


    if 'Modèlel' in df_res.columns:
        df_res = df_res.rename(columns = {
            'Modèlel': 'Modèle AIVAM'
        })

    return df_res


In [ ]:
#data dirs
cwd = os.getcwdb()
data_21 = f'BDD AIVAM FY 2021.xlsx'
dwh = f'AIVAM DWH.xlsx'
path_of_Net_models = 'Models Net.xlsx'

models_dict = {}

# get data segments and hybride types
df_aivam_segments = pd.read_excel(data_21, sheet_name = 'DATA_Segments')
df_aivam_hyb_type_mod = pd.read_excel(data_21, sheet_name = 'DATA_Type_hyb_mod')
df_aivam_transmission = pd.read_excel(data_21, sheet_name = 'DATA_Transmission_CSUV')
df_models = pd.read_excel(path_of_Net_models)
data_years = 'TDB//TdB_Novembre_2021 (1).xlsm'
cu_year = 'Base Brute AIVAM 202205.xlsx'

#Getting models Net into dict
for i in range(df_models.shape[0]):
    models_dict[df_models.loc[i, 'Modèle AIVAM']] = df_models.loc[i, 'Modèle Net']

df_years = pd.read_excel(data_years, sheet_name = 'DATA_Brut')

## process when extracting the previous years from 2014 tell now
df_per_year = []
year = dt.datetime.now().year
for year_index in range(2014,year):
    df_y =  df_years[df_years.Annee == year_index]
    df_y = rename_year(df_y)
    df_per_year.append(df_y)

df_cu_year = pd.read_excel(cu_year) #ms_df[ms_df.Year == year]
df_cu_year = rename_year(df_cu_year)
df_per_year.append(df_cu_year)
#ms_df = ms_df[ms_df.Year != year]

#df_cu_year_months = list(set(df_cu_year.Mois.values))
df_per_year_af_pro = []
for data_brut in df_per_year:
    data_now_res = global_process(data_brut, df_aivam_segments, 
        df_aivam_hyb_type_mod, df_aivam_transmission
    )
    df_per_year_af_pro.append(data_now_res)

data_cols = [
    'Year', 'Mois', 'City', 'Marque', 'Groupe',
    'Modèle AIVAM', 'CKD/CBU', 'VP/VUL', 'Caratéristiques',
    'CARBURANT', 'Sales', 'segmentation', 
    'Months', 'Transmission', 'Type Motorisation', 'Model Net',
    'Brand Order'
]

df_aivam_res = pd.concat(df_per_year_af_pro)

#df_aivam_res = rename_year(df_aivam_res)
all_cols = list(df_aivam_res.columns)

unwanted_cols = [col for col in all_cols if col not in data_cols]

df_aivam_res = df_aivam_res.drop(unwanted_cols, axis=1) 



In [ ]:
Boite_vitesse = []
for index_ in range(df_aivam_res.shape[0]):
    if ' BVA ' in df_aivam_res.iloc[index_]['Caratéristiques']:
        Boite_vitesse.append('A')
    elif ' BVM ' in df_aivam_res.iloc[index_]['Caratéristiques']:
        Boite_vitesse.append('M')
    elif ' BA ' in df_aivam_res.iloc[index_]['Caratéristiques']:
        Boite_vitesse.append('A')
    elif ' BM ' in df_aivam_res.iloc[index_]['Caratéristiques']:
        Boite_vitesse.append('M')
    else: 
        Boite_vitesse.append('None')

df_aivam_res['Transmission'] = Boite_vitesse

In [ ]:
transmission = df_aivam_transmission #transmission = pd.read_excel(tdb, sheet_name='DATA_Transmission_CSUV')
trans_dic = {}
for index_ in range(transmission.shape[0]):
    trans_dic[transmission.iloc[index_]['Caratéristiques']] = transmission.iloc[index_]['Transmission']

def get_transmission(x):
    if x['Transmission'] == 'None':
        if x['Caratéristiques'] in trans_dic.keys():
            return trans_dic[x['Caratéristiques']]
        else : 
            return 'None'
    else:
        return x['Transmission']

df_aivam_res['Transmission'] = df_aivam_res.apply(
    lambda x: get_transmission(x), axis = 1
)

In [ ]:
def get_bv_from_hyb(x):
    if x['Type Motorisation'] in ['HEV', 'EV', 'PHEV']:
        return 'A'
    else: 
        return x['Transmission']

df_aivam_res['Transmission'] = df_aivam_res.apply(
    lambda x: get_bv_from_hyb(x), axis = 1
)

In [ ]:
def ret_Cap(x):
    x = str(x)
    return x.upper()

In [ ]:
bv2 = 'c://user//SC97973//Modèles Office personnalisés//AIVAM BDD//Boite vitesse with BV2.xlsx'
df_bv2 = pd.read_excel(bv2)
df_bv2['Modele'] = df_bv2['Modele'].apply (lambda x: ret_Cap(x))

Auto_Models = []
Mann_Models = []
i = 0
for brand in set(df_bv2['Modele'].values) : #.shape[0]:
    df_brand = df_bv2[df_bv2.Modele == brand]
    if 'M' not in list(set(df_brand.Transmission.values)):
        Auto_Models.append(brand)
    if 'A' not in list(set(df_brand.Transmission.values)):
        Mann_Models.append(brand)
    #print(brand)


In [ ]:
def get_transmission_from_model(x):
    if x['Transmission'] == 'None':
        if x['Model Net'] in Auto_Models:
            return 'A'
        elif x['Model Net'] in Mann_Models:
            return 'M'
        else : 
            return 'None'
    else:
        return x['Transmission']
df_aivam_res['Model Net'] = df_aivam_res['Model Net'].apply(lambda x: ret_Cap(x))
df_aivam_res['Transmission'] = df_aivam_res.apply(
    lambda x: get_transmission_from_model(x), axis = 1
)

In [ ]:
df_aivam_res.columns

Index(['Year', 'Mois', 'Marque', 'Modèle AIVAM', 'Caratéristiques', 'CKD/CBU',
       'segmentation', 'Transmission', 'VP/VUL', 'CARBURANT',
       'Type Motorisation', 'Months', 'Sales', 'City', 'Groupe', 'Brand Order',
       'Model Net'],
      dtype='object')

In [ ]:
df_22 = df_aivam_res[df_aivam_res.Year == 2022]
print(df_22[df_22.Transmission == 'None']['Sales'].sum())
print(df_22[df_22.Transmission == 'A']['Sales'].sum())
print(df_22[df_22.Transmission == 'M']['Sales'].sum())
print(df_22['Sales'].sum())

27598
19308
20102
67008


In [ ]:

unnamed_cols = get_Unnamed_columns(df_aivam_res)
df_aivam_res = df_aivam_res.drop(unnamed_cols, axis=1)


writer = pd.ExcelWriter(dwh, engine = 'xlsxwriter')
df_aivam_res.to_excel(
    writer, 
    sheet_name = 'Market_Analysis', 
    #columns = list(df_aivam_res.columns), 
    index = False
)

writer.save()